## CREATING LAYERS : FORWARD AND BACKWARD PASS

In [1]:
import numpy as np

In [2]:
# Dense Layer
class Layer_Dense:
    # Layer initialization

    def __init__(self,n_inputs,n_neurons):
        # Initialize weights and biases

        self.weights = 0.01*np.random.randn(n_inputs,n_neurons)
        self.biases = np.zeros((1,n_neurons))


    # Forward pass

    def forward(self,inputs):
        # Remember input values

        self.inputs = inputs 
        # Calculate output values from input ones, weights and biases
        self.output = np.dot(inputs,self.weights) + self.biases
    
    # Backward pass

    def backward(self,dvalues):
        # Gradients on parameters

        self.dweights = np.dot(self.inputs.T,dvalues)
        self.dbiases = np.sum(dvalues,axis=0,keepdims=True)

        # Gradient on values

        self.dinputs = np.dot(dvalues,self.weights.T)




### RELU ACTIVATION: FORWARD AND BACKWARD PASS

In [3]:
# ReLU acdtiation

class Activation_ReLU:
    # Forward pass
    def forward(self,inputs):
        # Remember input values
        self.inputs = inputs
        # Calculate output values from inputs 

        self.output = np.maximum(0,inputs)


    # Backward pass

    def backward(self,dvalues):
        # Since we need to modify the original variable,
        # let's make a copy of values first
        self.dinputs = dvalues.copy()
        # Zero gradient where input values were negative
        self.dinputs[self.inputs <=0] = 0

# FULL CODE UPTO THIS POINT : FORWARD AND BACKWARD PASS

In [5]:
%run 10_Backpropagtation_Softmax_A_Cross_Entropy_loss.ipynb

[0.7 0.5 0.9]
[[0.35667494 2.30258509 1.60943791]
 [2.30258509 0.69314718 0.91629073]
 [3.91202301 0.10536052 2.52572864]]
0.38506088005216804
[0.35667494 0.69314718 0.10536052]
0.38506088005216804
0.38506088005216804
Gradients: combined loss and activation :
[[-0.1         0.03333333  0.06666667]
 [ 0.03333333 -0.16666667  0.13333333]
 [ 0.00666667 -0.03333333  0.02666667]]


In [4]:
import nnfs 
from nnfs.datasets import spiral_data  

In [6]:
# Create dataset

X,y = spiral_data(samples=100,classes=3)

# Create Dense layer with 2 input features and 3 output values 

dense1 = Layer_Dense(2,3)
# Create ReLU activation (to be used with Dense layer):
activation1 = Activation_ReLU()

# Create second Dense layer with 3 input features (as we take output
#of previous layer here)  and 3 output values 

dense2 = Layer_Dense(3,3)

# Create Softmax classifier's combined loss and activation 
loss_activation = Activation_Softmax_Loss_CategoricalCrossentropy()

# Perform a forward pass of our training data throght this layer
dense1.forward(X)

# Perform a forward pass throught activation function
# takes the output of first dense layer here


activation1.forward(dense1.output)

# Perform a forward pass throught second Dense layer
# takes outputs of actiavtion function of first layer as inputs

dense2.forward(activation1.output)

# Perform a forward pass through the actiavtion/loss functin
# akes the output of second dense alyer here and return loss

loss = loss_activation.forward(dense2.output,y)

# Let's see output of the first few samples:

print(loss_activation.output[:5])

# print loss values
print('loss',loss)



[[0.33333333 0.33333333 0.33333333]
 [0.33333264 0.3333343  0.33333305]
 [0.3333318  0.33333549 0.33333271]
 [0.33333109 0.33333649 0.33333242]
 [0.33333055 0.33333726 0.33333219]]
loss 1.0986023939283553


In [7]:
# Calculate accuracy from output of activation2 and targets 
# calculate values along first axis

predictions = np.argmax(loss_activation.output, axis=1)

if len(y.shape) == 2:
    y = np.argmax(y,axis=1)

accuracy = np.mean(predictions == y)

# Print accuracy

print('acc:',accuracy)

# Backward pass

loss_activation.backward(loss_activation.output,y)
dense2.backward(loss_activation.dinputs)
activation1.backward(dense2.dinputs)
dense1.backward(activation1.dinputs)

# Print gradients

print(dense1.dweights)
print(dense1.dbiases)
print(dense2.dweights)
print(dense2.dbiases)

acc: 0.38333333333333336
[[-1.66607232e-04  1.19496997e-04  1.54812099e-04]
 [ 3.40522376e-04  8.09968669e-06 -4.89209939e-04]]
[[-1.25220687e-04 -4.84606863e-05 -5.79296184e-04]]
[[-9.54183094e-06  5.20428070e-05 -4.25009761e-05]
 [-7.08412154e-05  1.61847161e-04 -9.10059452e-05]
 [ 8.99390193e-05 -4.59010860e-04  3.69071841e-04]]
[[-1.34145411e-05  1.65809705e-05 -3.16642941e-06]]
